# Clustering Exploration 

Experimenting with clustering Galveston Indoor/Outdoor points to get routes for each day. 

In [31]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv

In [32]:
trucks = pd.read_csv('../data/truck_service.csv')
trucks.head()

,Name,Longitude,Latitude,Daily_Pickup_Totes,Weekly_Dropoff_Totes,user_ratings_total,category,rating,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,location_type,id,pickup_type
0,Moody Gardens,-94.852300,29.273600,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,depot,NaN,NaN
1,Sugar Bean,-94.870418,29.261029,1.0,1.0,476.0,cafe,4.7,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,indoor,NaN,NaN
2,MAC-IES,-94.791385,29.298912,1.0,1.0,73.0,restaurant,5.0,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,indoor,NaN,NaN
3,Good Dough,-94.808531,29.290612,1.0,1.0,68.0,bakery,4.6,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,indoor,NaN,NaN
4,Porch Cafe,-94.750192,29.319581,1.0,1.0,332.0,restaurant,4.1,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,indoor,NaN,NaN


In [33]:
X = trucks[['Name', 'Longitude', 'Latitude']]

kmeans = KMeans(n_clusters = 5, init ='k-means++')
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['cluster_number'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point
X.head(10)

/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/tmp/ipykernel_92/4231203079.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cluster_number'] = kmeans.fit_predict(X[X.columns[1:3]])


,Name,Longitude,Latitude,cluster_number
0,Moody Gardens,-94.852300,29.273600,2
1,Sugar Bean,-94.870418,29.261029,2
2,MAC-IES,-94.791385,29.298912,0
3,Good Dough,-94.808531,29.290612,4
4,Porch Cafe,-94.750192,29.319581,0
5,Coastal Grill,-94.789455,29.308158,0
6,Texas A&M University at Galveston,-94.818287,29.315497,0
7,Rudy & Paco Restaurant and Bar,-94.791081,29.305336,0
8,Vargas Cut and Catch (listed above),-94.791385,29.305227,0
9,MarMo Cafe & Lounge,-94.792103,29.305600,0


In [34]:
X.cluster_number.unique()

array([2, 0, 4, 1, 3], dtype=int32)

In [35]:
X['cluster_number'].value_counts()

0    119
4     75
2     35
1      9
3      5
Name: cluster_number, dtype: int64

In [36]:
def redistribute_clusters(X, kmeans, max_iterations=100):
    for _ in range(max_iterations):
        cluster_counts = X['cluster_number'].value_counts().sort_index()
        min_cluster_size = cluster_counts.min()
        max_cluster_size = cluster_counts.max()
        
        # Find the cluster with the maximum size and the one with the minimum size
        max_cluster = cluster_counts.idxmax()
        min_cluster = cluster_counts.idxmin()
        
        # Get the points from the maximum cluster
        max_cluster_points = X[X['cluster_number'] == max_cluster]
        
        # Find the point in the maximum cluster that is closest to the centroid of the minimum cluster
        min_cluster_center = kmeans.cluster_centers_[min_cluster]
        closest_point = max_cluster_points.apply(lambda row: np.linalg.norm(row[['Longitude', 'Latitude']] - min_cluster_center), axis=1).idxmin()
        
        # Reassign the closest point from the maximum cluster to the minimum cluster
        X.at[closest_point, 'cluster_number'] = min_cluster
        
    return X

clusteredPoints = redistribute_clusters(X, kmeans, max_iterations=100)


In [41]:
clusteredPoints['cluster_number'].value_counts()
clustered_data = trucks.merge(clusteredPoints, left_on='Name', right_on='Name')
clustered_data = clustered_data.drop(['Longitude_y', 'Latitude_y'], axis=1)

In [42]:
clustered_data

,Name,Longitude_x,Latitude_x,Daily_Pickup_Totes,Weekly_Dropoff_Totes,user_ratings_total,category,rating,weekly_high,weekly_low,...,Address,geometry,Adoption,Adoption Guess,Visited,Influence,location_type,id,pickup_type,cluster_number
0,Moody Gardens,-94.852300,29.273600,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,depot,NaN,NaN,2
1,Sugar Bean,-94.870418,29.261029,1.0,1.0,476.0,cafe,4.7,200,50,...,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,indoor,NaN,NaN,2
2,MAC-IES,-94.791385,29.298912,1.0,1.0,73.0,restaurant,5.0,1500,1050,...,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,indoor,NaN,NaN,3
3,Good Dough,-94.808531,29.290612,1.0,1.0,68.0,bakery,4.6,70,35,...,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,indoor,NaN,NaN,4
4,Porch Cafe,-94.750192,29.319581,1.0,1.0,332.0,restaurant,4.1,300,NaN,...,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,indoor,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,NaN,-94.956086,29.196348,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,outdoor,3034.0,Truck,2
5351,NaN,-94.956086,29.196348,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,outdoor,3034.0,Truck,2
5352,NaN,-94.956086,29.196348,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,outdoor,3034.0,Truck,1
5353,NaN,-94.956086,29.196348,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,outdoor,3034.0,Truck,1


In [43]:
clustered_data.to_csv ('../data/clustered_data.csv', index=None, header = True)